In [ ]:
import os
import pandas as pd
from glob import glob
from tqdm import tqdm
from time import time

import src as sp

In [ ]:
# parameters
pdb_filepaths = glob("tmp/*.pdb")
output_dir = "tmp_mpnn_out"

In [ ]:
# load structure
profiling = []
for pdb_filepath in tqdm(pdb_filepaths):
    pdbid = os.path.basename(pdb_filepath)[:-4]
    
    # find subunits
    subunits = sp.split_by_chain(sp.read_pdb(pdb_filepath))
    cids = [cid.split(':')[0] for cid in subunits]
    
    # run ProteinMPNN
    command = ' '.join([
        "python", "ProteinMPNN/protein_mpnn_run.py",
        "--pdb_path", pdb_filepath,
        "--out_folder", output_dir,
        "--num_seq_per_target", "1",
        "--sampling_temp", "0.000001",
        "--seed", "37",
        "--batch_size", "1",
    ])
    output = os.popen(command).read()
    
    # save profiling
    sid = os.path.basename(pdb_filepath).split('.')[0]
    profiling.append({'sid': sid, 'dt': float(output.split()[-2])})

# save profiling
dfp = pd.DataFrame(profiling)
dfp.to_csv("results/profiling_mpnn_ext.csv", index=False)